# Autoregressive Integrated Moving Average (ARIMA)

### Importing Required Libraries

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta

from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from time import time
import seaborn as sns
sns.set_theme(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')


### Importing Data Files

In [54]:
bsp_data = pd.read_pickle(r"C:\Users\91780\Desktop\IIT bhilai internship\Code\Githubrepocode.ipynb\Dataset\clean_06_02_24.pkl")
region_wise = pd.read_pickle(r"C:\Users\91780\Desktop\IIT bhilai internship\Code\Githubrepocode.ipynb\Dataset\regionwise_columns.pickle")
df = bsp_data[region_wise['Stand_13-18']]

### Longest Continous Interval

In [55]:
# converting the index to date time
df.index = pd.to_datetime(df.index)

df = df.loc['2024-02-06 04:41:15.920000':'2024-02-06 05:25:58.950000']
df.head()

,[9.226],[12:44],[9:12],[9:13],[9:14],[9:15],[9:16],[9:17],[9:47],[9:48],...,[12:65],[12:66],[12:67],[12:68],[11:21],[11:22],[11:23],[11:24],[11:25],[11:26]
Time,,,,,,,,,,,,,,,,,,,,,
2024-02-06 04:41:15.920,0,84,23,4,34,39,41,29,981,1539,...,404,288,483,767,1.132,1.085,1.15,1.157,1.208,1.168
2024-02-06 04:41:16.920,0,84,23,7,34,40,41,29,983,1542,...,365,306,488,797,1.132,1.085,1.15,1.157,1.208,1.168
2024-02-06 04:41:17.920,0,84,24,4,34,39,41,29,983,1542,...,364,292,481,752,1.132,1.085,1.15,1.157,1.208,1.168
2024-02-06 04:41:18.920,0,84,24,8,36,40,41,29,982,1540,...,385,304,484,757,1.132,1.085,1.15,1.157,1.208,1.168
2024-02-06 04:41:19.920,0,84,23,4,33,39,42,29,982,1540,...,377,276,490,783,1.132,1.085,1.15,1.157,1.208,1.168


### Stand 16 current

In [56]:
# Sensor in consideration
col = '[12:42]'

# Data visualization
fig = go.Figure()

fig.add_trace(go.Scatter(x=df[col].index,
                         y=df[col],
                         mode='lines',
                         name="Data"))
fig.update_layout(
    title=f'Time Series of Stand 13 current',
    xaxis_title='time',
    yaxis_title='[12:42]',
    showlegend=True
)

fig.show()

### Check for stationarity

In [57]:
def ad_test(df):
    dftest = adfuller(df, autolag = 'AIC')
    for x in dftest:
        print("\t",x)
    if(dftest[1] <= 0.05):
        print("Null Hypothesis rejeted")
    else:
        print("Weak evidence against Null Hypothesis, time series has unit root, indicating it is non-stationary")

In [58]:
ad_test(df[col])

	 -17.730988442064966
	 3.4350008213846025e-30
	 28
	 2655
	 {'1%': -3.4328153987492263, '5%': -2.8626292280252916, '10%': -2.567349833523076}
	 29097.959615554824
Null Hypothesis rejeted


### Order of ARIMA

In [59]:
# using auto-arima from pmdarima library
stepwise_fit=auto_arima(df[col],
                        start_p=0,start_q=0,
                        max_p=5,max_q=5,
                        seasonal=True, 
                        trace=True,
                        series=True)

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=31722.873, Time=0.22 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=29713.714, Time=0.31 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=30417.340, Time=2.98 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=40175.117, Time=0.09 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=29715.672, Time=0.71 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=29715.691, Time=0.95 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=29717.721, Time=3.43 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=30094.854, Time=0.09 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 8.955 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 2684
Model:               SARIMAX(1, 0, 0)   Log Likelihood              -14853.857
Date:                Thu, 04 Jul 2024   AIC                          29713.714
Time:                        12:16:57   BIC                          29731.399
Sample:                    02-06-2024   HQIC                         29720.112
                         - 02-06-2024                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    115.2717      3.534     32.621      0.000     108.346     122.197
ar.L1          0.7263      0.008     86.135      0.000       0.710       0.743
sigma2      3754.0742     43.796     85.718      0.000    3668.236    3839.912
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):             25048.97
Prob(Q):                              0.88   Prob(JB):                         0.00
Heteroskedasticity (H):               1.31   Skew:                             0.43
Prob(H) (two-sided):                  0.00   Kurtosis:                        17.94
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Model Fitting

#### Splitting into Training and Testing set

In [70]:
# Splitting into 80% training and testing 20% set
split_val=int(len(df[col])*0.8)

# Training set
df['train'] = df[col].iloc[:split_val]

# Testing set
df['test'] = df[col].iloc[split_val:]

print(len(df[col]), split_val, df['train'].shape, df['test'].shape)

2684 2147 (2684,) (2684,)


##### Plotting training and testing set

In [72]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df[col].index,
                         y=df[col],
                         mode='lines',
                         name="Actual"))

fig.add_trace(go.Scatter(x=df[col].index,
                         y=df['train'],
                         mode='lines',
                         name="training set"))

fig.add_trace(go.Scatter(x=df[col].index,
                         y=df['test'],
                         mode='lines',
                         name="testing set"))

fig.update_layout(
    title=f'Actual/Predicted data stand 16 current',
    xaxis_title='time',
    yaxis_title='[12:42]',
    showlegend=True
)

fig.show()

#### Model training

In [75]:
# Idea: Train the model for training set. Forecast the values for future 5 timestamps. Store the forecasted value. Again
#       train the model for the length equal to training set(do this by removing 5 seconds data from previous set and adding
#       new 5 seconds data from testing set). 
# Example: 1st iteration Train -> 0 - 100 sec    Test -> 20 sec                       
#          2nd iteration Train -> 5 - 105 sec    Test -> 15 sec  
#          3rd iteration Train -> 10 - 110 sec    Test -> 10 sec  

itr=0

# List to store the forecasted values
y_pred = [] 
history_endog = list(df['train'].copy(deep=True))

while itr < len(test.iloc[:150]): 
    # training of model
    sarima = SARIMAX(history_endog,order=(1,0,0),seasonal_order=(1,0,0,40))
    loop_train_model = sarima.fit()

    # Forecasting values for future 5 timestamps (here in seconds)
    forecasted = loop_train_model.forecast(steps=5)
    print(forecasted)

    # Storing the forecasted value in y_pred
    y_pred.extend(list(forecasted))

    # To train the model for fixed length each time. (here length = len(train)) 
    history_endog = history_endog[5:] 
    history_endog.extend(list(test[:5])) # Extending history_endog with actual data  
    test = test.iloc[5:]
    itr+=5

[0.47412145 0.46874925 0.46343793 0.4581868  0.45299516]
[456.80103863 451.66057606 446.577954   441.55252155 436.58363516]
[454.82155649 449.70140312 444.63888369 439.63334938 434.68415869]
[206.59341285 204.21454045 201.86306368 199.53866708 197.24103883]
[475.51245406 470.08754526 464.72455901 459.42278886 454.18153644]
[438.89061976 433.84001057 428.84749612 423.91240787 419.03408496]
[437.91508732 432.88849114 427.91954209 423.00757846 418.15194612]
[441.08590343 436.22512108 431.41706543 426.66115533 421.95681592]
[446.93129102 441.9574435  437.00894357 432.11915558 427.31767495]
[457.7964626  452.71807994 447.69271895 442.61062487 437.74586389]
[444.99373876 439.97605767 435.95111215 431.08868511 426.33295894]
[493.48493136 488.01425026 482.61238628 476.36650495 470.82830116]
[204.61329593 202.38134961 199.97765858 197.68311338 195.47491338]
[470.61769488 465.14730976 459.9626347  454.71558623 449.64912882]
[454.6847041  449.4825528  444.33219871 439.24813012 434.25242579]
[458.

#### Plotting the forecasted value and actual value for comparison

In [76]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df[col].index,
                         y=test,
                         mode='lines',
                         name="Actual"))

fig.add_trace(go.Scatter(x=df[col].index,
                         y=y_pred,
                         mode='lines',
                         name="Predicted"))

fig.update_layout(
    title=f'Actual/Predicted data stand 16 current',
    xaxis_title='time',
    yaxis_title='[12:42]',
    showlegend=True
)

fig.show()